In [1]:

import os
import sys
import random
import logging
import rasterio
import argparse
import numpy as np
from tqdm import tqdm
from os.path import dirname as up

import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

path_cur = os.path.abspath(os.getcwd())

sys.path.append(path_cur)
from unet import UNet
from vims_dataloader import GenDEBRIS, bands_mean, bands_std

sys.path.append(os.path.join(up(up(path_cur)), 'utils'))

from vims_metrics import Evaluation, confusion_matrix
# from vims_assets import labels, labels_conf
from pathlib import Path

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

root_path = up(up(path_cur))

logging.basicConfig(filename=os.path.join(root_path, 'logs','evaluating_unet.log'), filemode='a',level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')
logging.info('*'*10)

from torchgeo.trainers import SemanticSegmentationTask
from pytorch_lightning import Trainer, seed_everything
import pytorch_lightning as pl
from typing import Any, Dict, cast
from torchmetrics import Accuracy, JaccardIndex, MetricCollection, Precision, Recall
import matplotlib.pyplot as plt
from vims_segmentation import SemanticSegmentation
from processing_utils import ensemble


In [60]:
def main(options):

    # Transformations
    transform_test = transforms.Compose([transforms.ToTensor()])
    standardization = transforms.Normalize(bands_mean, bands_std)
    
    # Construct Test data loader
    dataset_test = GenDEBRIS('test', transform=transform_test, standardization = standardization, agg_to_water = options['agg_to_water'])
    test_loader = DataLoader(   dataset_test, 
                                batch_size = options['batch'], 
                                shuffle = False)
    
    
    # Load pretrained model
    model_name = os.path.basename(options["model_path"])
        
    if "CW" in model_name: # and "deeplabv3+" not in model_name
        model = SemanticSegmentation.load_from_checkpoint(options["model_path"])
        model.eval()
    else:
        model = SemanticSegmentationTask.load_from_checkpoint(options["model_path"])
        model.eval()
    
    
    
    y_true = []
    y_predicted = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="testing"):
            
            image = batch["image"]
            target = batch["mask"]

            logits = model(image) # predicted
            logits = torch.nn.functional.softmax(logits, dim=1).argmax(1)
            
            logits = logits.reshape(-1)
            target = target.reshape(-1)

            # Accuracy metrics only on annotated pixels (Check on the dimension changes)

            mask = target != 0 # 0 is the background
            
            logits = logits[mask]
            target = target[mask]

            y_predicted += logits.tolist()
            y_true += target.tolist()
            
        
        ####################################################################
        # Save Scores to the .log file                                     #
        ####################################################################
        acc = Evaluation(y_predicted, y_true)
        logging.info("\n")
        logging.info("STATISTICS: \n")
        logging.info("Evaluation: " + str(acc))
        print("Evaluation: " + str(acc))
        """
        Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
        You being i-th class and predicted label being j-th class.
        """
        conf_mat = confusion_matrix(y_true, y_predicted, options['labels'])
        
        logging.info("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
 

        if options['predict_masks']:
            
            path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test')
            ROIs = [file for file in os.listdir(path) if file.split('.')[-1]=='tif']

            impute_nan = np.tile(bands_mean, (256,256,1))
                        
            for roi in tqdm(ROIs):
                
                roi_file = os.path.join(path, roi)
                os.makedirs(options['gen_masks_path'], exist_ok=True)
            
                output_image = os.path.join(options['gen_masks_path'], os.path.basename(roi_file).split('.tif')[0] + '_unet.tif')
            
                # Read metadata of the initial image
                with rasterio.open(roi_file, mode ='r') as src:
                    tags = src.tags().copy()
                    meta = src.meta
                    image = src.read()
                    image = np.moveaxis(image, (0, 1, 2), (2, 0, 1)).astype('float32')
                    dtype = src.read(1).dtype
            
                # Update meta to reflect the number of layers
                meta.update(count = 1)
            
                # Write it
                with rasterio.open(output_image, 'w', **meta) as dst:
                    
                    # Preprocessing before prediction
                    nan_mask = np.isnan(image)
                    image[nan_mask] = impute_nan[nan_mask]
                
                    image = transform_test(image)
                    image = standardization(image)

                    # Predictions
                    logits = model(image.unsqueeze(0))
                    probs = logits.softmax(dim=1).numpy().argmax(1).squeeze()
                    
                    # Write the mask with georeference
                    dst.write_band(1, probs.astype(dtype).copy()) # In order to be in the same dtype
                    dst.update_tags(**tags)


In [36]:
# def main_ensemble(options):
    
#     # Transformations
#     transform_test = transforms.Compose([transforms.ToTensor()])
#     standardization = transforms.Normalize(bands_mean, bands_std)
    
#     # Construct Test data loader
#     dataset_test = GenDEBRIS('val', transform=transform_test, standardization = standardization, agg_to_water = options['agg_to_water'])
#     test_loader = DataLoader(   dataset_test, 
#                                 batch_size = options['batch'], 
#                                 shuffle = False)
    
#     models = options["model_paths"]
    
#     y_true = []
#     y_predicted = []
    
    
#     with torch.no_grad():
#         for batch in tqdm(test_loader, desc="testing"):
            
#             y_temp_predicted = []
                
#             for model in models:
#                 print('Working on model {}'.format(model))
#                 # Load pretrained model
#                 model = SemanticSegmentationTask.load_from_checkpoint(options["model_path"])    
#                 model.eval()

#                 image = batch["image"]
#                 target = batch["mask"]

#                 logits = model(image) # predicted
#                 logits = torch.nn.functional.softmax(logits, dim=1).argmax(1)

#                 logits = logits.reshape(-1)
#                 target = target.reshape(-1)

#                 # Accuracy metrics only on annotated pixels (Check on the dimension changes)

#                 mask = target != 0 # 0 is the background

#                 logits = logits[mask]
#                 target = target[mask]
                
#                 y_temp_predicted.append(logits.tolist())
            
#             y_temp_predicted = np.array(y_temp_predicted)
#             y_pr_ensemble = np.apply_along_axis(ensemble, axis=0, arr=y_temp_predicted).tolist()
#             y_predicted += y_pr_ensemble
#             y_true += target.tolist()
            
#         ####################################################################
#         # Save Scores to the .log file                                     #
#         ####################################################################
#         acc = Evaluation(y_predicted, y_true)
#         logging.info("\n")
#         logging.info("STATISTICS: \n")
#         logging.info("Evaluation: " + str(acc))
#         print("Evaluation: " + str(acc))
#         """
#         Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
#         You being i-th class and predicted label being j-th class.
#         """
        
#         conf_mat = confusion_matrix(y_true, y_predicted, options['labels'])
        
#         logging.info("Confusion Matrix:  \n" + str(conf_mat.to_string()))
#         print("Confusion Matrix:  \n" + str(conf_mat.to_string()))




In [37]:

# model_dir = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results"
# model_files = [file for file in os.listdir(model_dir) if file.split(".")[-1]=="ckpt" and file.split("_")[0]=="4class"]

# gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')
# labels = ['Background', 'Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']

# options = {'agg_to_water': True, 'batch': 32, 
#               'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
#               'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels,
#               'model_paths': model_files}

# main_ensemble(options)


In [10]:
# model_path = os.path.join(path_cur, 'trained_models', '200', 'model.pth')

model_path = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_norm/4class_deeplabv3+_resnet50_0.001_ce_imagenet_CWTrue-epoch=47-val_loss=0.58.ckpt"

gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')
labels = ['Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']

options = {'agg_to_water': True, 'batch': 64, 
          'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
          'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path,
          'labels': labels}

main(options)

In [ ]:
model_dir = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_norm"
model_files = [file for file in os.listdir(model_dir) if file.split(".")[-1]=="ckpt" and file.split("_")[0]=="4class"]

gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')

for file in model_files:
    print("Using model {}".format(file))
    model_path = os.path.join(model_dir, file)
    labels = ['Background', 'Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
    options = {'agg_to_water': True, 'batch': 32, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
    try:
        main(options)
    except:
        labels = ['Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
        options = {'agg_to_water': True, 'batch': 32, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
        main(options)


Using model 4class_unet_resnet18_0.001_ce_swsl-epoch=47-val_loss=0.60.ckpt


testing: 100%|██████████| 74/74 [00:35<00:00,  2.11it/s]


Evaluation: {'macroPrec': 0.4862221445339181, 'microPrec': 0.732898287525664, 'weightPrec': 0.7416448237086782, 'macroRec': 0.6260572442821115, 'microRec': 0.732898287525664, 'weightRec': 0.732898287525664, 'macroF1': 0.5273157404639642, 'microF1': 0.732898287525664, 'weightF1': 0.7354250236077897, 'subsetAcc': 0.732898287525664, 'IoU': 0.3872575694813679}
[[1351000.0, 536776.0, 3885.0, 15263.0, 1906924.0, 0.71], [383978.0, 1350456.0, 11680.0, 24796.0, 1770910.0, 0.76], [274.0, 1789.0, 4042.0, 3562.0, 9667.0, 0.42], [594.0, 7846.0, 1147.0, 15318.0, 24905.0, 0.62], [1735846.0, 1896867.0, 20754.0, 58939.0, 'mPA:', 0.63], [0.59, 0.58, 0.15, 0.22, 'mIoU:', 0.39], [0.78, 0.71, 0.19, 0.26, 'OA:', 0.73], [0.74, 0.74, 0.27, 0.37, 'F1-macro:', 0.53]]


testing: 100%|██████████| 74/74 [00:46<00:00,  1.58it/s]


Evaluation: {'macroPrec': 0.4862221445339181, 'microPrec': 0.732898287525664, 'weightPrec': 0.7416448237086782, 'macroRec': 0.6260572442821115, 'microRec': 0.732898287525664, 'weightRec': 0.732898287525664, 'macroF1': 0.5273157404639642, 'microF1': 0.732898287525664, 'weightF1': 0.7354250236077897, 'subsetAcc': 0.732898287525664, 'IoU': 0.3872575694813679}
[[1351000.0, 536776.0, 3885.0, 15263.0, 1906924.0, 0.71], [383978.0, 1350456.0, 11680.0, 24796.0, 1770910.0, 0.76], [274.0, 1789.0, 4042.0, 3562.0, 9667.0, 0.42], [594.0, 7846.0, 1147.0, 15318.0, 24905.0, 0.62], [1735846.0, 1896867.0, 20754.0, 58939.0, 'mPA:', 0.63], [0.59, 0.58, 0.15, 0.22, 'mIoU:', 0.39], [0.78, 0.71, 0.19, 0.26, 'OA:', 0.73], [0.74, 0.74, 0.27, 0.37, 'F1-macro:', 0.53]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall    Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall            1351000.0   536776.0   3885.0    15263.0  1906924.0   0.71
Rip Rap                          383978.0  135

...

testing: 100%|██████████| 74/74 [01:08<00:00,  1.08it/s]


Evaluation: {'macroPrec': 0.5367862514337375, 'microPrec': 0.7471529245454296, 'weightPrec': 0.7512840881116136, 'macroRec': 0.7016947517450354, 'microRec': 0.7471529245454296, 'weightRec': 0.7471529245454296, 'macroF1': 0.5898577180445089, 'microF1': 0.7471529245454295, 'weightF1': 0.7486959075135213, 'subsetAcc': 0.7471529245454296, 'IoU': 0.43720311425549274}
[[1442491.0, 451985.0, 3196.0, 9252.0, 1906924.0, 0.76], [430438.0, 1309164.0, 10316.0, 20992.0, 1770910.0, 0.74], [294.0, 904.0, 6707.0, 1762.0, 9667.0, 0.69], [719.0, 7613.0, 1200.0, 15373.0, 24905.0, 0.62], [1873942.0, 1769666.0, 21419.0, 47379.0, 'mPA:', 0.7], [0.62, 0.59, 0.28, 0.27, 'mIoU:', 0.44], [0.77, 0.74, 0.31, 0.32, 'OA:', 0.75], [0.76, 0.74, 0.43, 0.43, 'F1-macro:', 0.59]]


testing: 100%|██████████| 74/74 [01:15<00:00,  1.02s/it]


Evaluation: {'macroPrec': 0.5367862514337375, 'microPrec': 0.7471529245454296, 'weightPrec': 0.7512840881116136, 'macroRec': 0.7016947517450354, 'microRec': 0.7471529245454296, 'weightRec': 0.7471529245454296, 'macroF1': 0.5898577180445089, 'microF1': 0.7471529245454295, 'weightF1': 0.7486959075135213, 'subsetAcc': 0.7471529245454296, 'IoU': 0.43720311425549274}
[[1442491.0, 451985.0, 3196.0, 9252.0, 1906924.0, 0.76], [430438.0, 1309164.0, 10316.0, 20992.0, 1770910.0, 0.74], [294.0, 904.0, 6707.0, 1762.0, 9667.0, 0.69], [719.0, 7613.0, 1200.0, 15373.0, 24905.0, 0.62], [1873942.0, 1769666.0, 21419.0, 47379.0, 'mPA:', 0.7], [0.62, 0.59, 0.28, 0.27, 'mIoU:', 0.44], [0.77, 0.74, 0.31, 0.32, 'OA:', 0.75], [0.76, 0.74, 0.43, 0.43, 'F1-macro:', 0.59]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall    Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall            1442491.0   451985.0   3196.0     9252.0  1906924.0   0.76
Rip Rap                          430438.0  

testing:  35%|███▌      | 26/74 [00:23<00:43,  1.11it/s]

In [41]:
# Check training accuracy


model_dir = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_norm"
model_files = [file for file in os.listdir(model_dir) if file.split(".")[-1]=="ckpt" and file.split("_")[0]=="4class"]

gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')

for file in model_files:
    print("Using model {}".format(file))
    model_path = os.path.join(model_dir, file)
    labels = ['Background', 'Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
    options = {'agg_to_water': True, 'batch': 32, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
    try:
        main(options)
    except:
        labels = ['Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
        options = {'agg_to_water': True, 'batch': 32, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
        main(options)


Using model 4class_unet_resnet50_0.0001_ce_imagenet-epoch=25-val_loss=0.58.ckpt


testing: 100%|██████████| 76/76 [01:29<00:00,  1.18s/it]
/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation: {'macroPrec': 0.727291756319779, 'microPrec': 0.9343540093963577, 'weightPrec': 0.9345816345077772, 'macroRec': 0.7437132059857492, 'microRec': 0.9343540093963577, 'weightRec': 0.9343540093963577, 'macroF1': 0.735223929011848, 'microF1': 0.9343540093963577, 'weightF1': 0.9343213514577238, 'subsetAcc': 0.9343540093963577, 'IoU': 0.6813384474903345}
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [8.0, 1868172.0, 96858.0, 171.0, 2315.0, 1967524.0, 0.95], [0.0, 129634.0, 1724134.0, 10757.0, 16481.0, 1881006.0, 0.92], [2.0, 525.0, 5189.0, 64445.0, 2142.0, 72303.0, 0.89], [4.0, 3200.0, 5333.0, 1175.0, 240220.0, 249932.0, 0.96], [14.0, 2001531.0, 1831514.0, 76548.0, 261158.0, 'mPA:', 0.74], [0.0, 0.89, 0.87, 0.76, 0.89, 'mIoU:', 0.68], [0.0, 0.93, 0.94, 0.84, 0.92, 'OA:', 0.93], [0.0, 0.94, 0.93, 0.87, 0.94, 'F1-macro:', 0.74]]
Confusion Matrix:  
                     Background Bulkhead Or Sea Wall    Rip Rap    Groin Breakwater        Sum Recall
Background                  0.0           

testing: 100%|██████████| 76/76 [01:23<00:00,  1.10s/it]


Evaluation: {'macroPrec': 0.8526565169307672, 'microPrec': 0.8649178268255344, 'weightPrec': 0.8646426977766362, 'macroRec': 0.8197300610270029, 'microRec': 0.8649178268255344, 'weightRec': 0.8649178268255344, 'macroF1': 0.8337894278893294, 'microF1': 0.8649178268255344, 'weightF1': 0.864499842120296, 'subsetAcc': 0.8649178268255344, 'IoU': 0.7220002343461123}
[[1740825.0, 222976.0, 720.0, 3003.0, 1967524.0, 0.88], [267888.0, 1591935.0, 5989.0, 15194.0, 1881006.0, 0.85], [1258.0, 14724.0, 45687.0, 10634.0, 72303.0, 0.63], [5912.0, 9920.0, 5178.0, 228922.0, 249932.0, 0.92], [2015883.0, 1839555.0, 57574.0, 257753.0, 'mPA:', 0.82], [0.78, 0.75, 0.54, 0.82, 'mIoU:', 0.72], [0.86, 0.87, 0.79, 0.89, 'OA:', 0.86], [0.87, 0.86, 0.7, 0.9, 'F1-macro:', 0.83]]


...

testing: 100%|██████████| 76/76 [01:49<00:00,  1.45s/it]


Evaluation: {'macroPrec': 0.8502777965204364, 'microPrec': 0.8501694533257088, 'weightPrec': 0.8532267948505785, 'macroRec': 0.7980203725543278, 'microRec': 0.8501694533257088, 'weightRec': 0.8501694533257088, 'macroF1': 0.8179662677870875, 'microF1': 0.8501694533257088, 'weightF1': 0.8497674605517027, 'subsetAcc': 0.8501694533257088, 'IoU': 0.699834721312553}
[[1606466.0, 358086.0, 959.0, 2013.0, 1967524.0, 0.82], [191114.0, 1670137.0, 4532.0, 15223.0, 1881006.0, 0.89], [953.0, 15295.0, 41750.0, 14305.0, 72303.0, 0.58], [3979.0, 14789.0, 3660.0, 227504.0, 249932.0, 0.91], [1802512.0, 2058307.0, 50901.0, 259045.0, 'mPA:', 0.8], [0.74, 0.74, 0.51, 0.81, 'mIoU:', 0.7], [0.89, 0.81, 0.82, 0.88, 'OA:', 0.85], [0.85, 0.85, 0.68, 0.89, 'F1-macro:', 0.82]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall    Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall            1606466.0   358086.0    959.0     2013.0  1967524.0   0.82
Rip Rap                          19111

testing: 100%|██████████| 76/76 [01:22<00:00,  1.09s/it]


Evaluation: {'macroPrec': 0.8180615419406931, 'microPrec': 0.7957489813019913, 'weightPrec': 0.8007892365233613, 'macroRec': 0.7588686879640733, 'microRec': 0.7957489813019913, 'weightRec': 0.7957489813019913, 'macroF1': 0.7770179336339944, 'microF1': 0.7957489813019915, 'weightF1': 0.7950228728449272, 'subsetAcc': 0.7957489813019913, 'IoU': 0.6405591809711537}
[[1470503.0, 478857.0, 1050.0, 17114.0, 1967524.0, 0.75], [272824.0, 1582521.0, 1881.0, 23780.0, 1881006.0, 0.84], [362.0, 21181.0, 38969.0, 11791.0, 72303.0, 0.54], [2086.0, 17802.0, 3155.0, 226889.0, 249932.0, 0.91], [1745775.0, 2100361.0, 45055.0, 279574.0, 'mPA:', 0.76], [0.66, 0.66, 0.5, 0.75, 'mIoU:', 0.64], [0.84, 0.75, 0.86, 0.81, 'OA:', 0.8], [0.79, 0.79, 0.66, 0.86, 'F1-macro:', 0.78]]


FileNotFoundError: [Errno 2] No such file or directory: '/rapids/notebooks/sciclone/geograd/Miranda/VIMS/NAIP/VA_NAIP_2018_8977/train'

In [ ]:
# # model_path = os.path.join(path_cur, 'trained_models', '200', 'model.pth')

# model_path = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results/4class_unet_resnet50_0.0001_jaccard_imagenet-epoch=95-val_loss=0.52.ckpt"
# #4class_unet_resnet18_0.0001_ce_imagenet-epoch=15-val_loss=0.63.ckpt
# #4class_unet_resnet18_0.0001_jaccard_imagenet-epoch=67-val_loss=0.53.ckpt
# #4class_unet_resnet18_0.0001_jaccard_ssl-epoch=77-val_loss=0.52.ckpt
# #7class_unet_resnet18_0.0001_ce_imagenet-epoch=10-val_loss=0.84.ckpt
# gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')


# options = {'agg_to_water': True, 'batch': 32, 
#           'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
#           'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path}

# main(options)

In [59]:
# model_path = os.path.join(path_cur, 'trained_models', '200', 'model.pth')

model_path_2 = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_batch32/4class_unet_resnet50_0.001_ce_imagenet_CWFalse_(0.9, 0.999)-epoch=55-val_loss=0.54.ckpt"

gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')
labels = ['Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']

options = {'agg_to_water': True, 'batch': 32, 
          'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
          'model_path': model_path_2, 'predict_masks': False, 'gen_masks_path':gen_masks_path,
          'labels': labels}

main(options)

testing: 100%|██████████| 23/23 [00:23<00:00,  1.04s/it]


Evaluation: {'macroPrec': 0.7064169812245192, 'microPrec': 0.7623679008639017, 'weightPrec': 0.7624742900009198, 'macroRec': 0.6578357397054484, 'microRec': 0.7623679008639017, 'weightRec': 0.7623679008639017, 'macroF1': 0.6795330993427686, 'microF1': 0.7623679008639017, 'weightF1': 0.7617965706184571, 'subsetAcc': 0.7623679008639017, 'IoU': 0.5293647620251909}
[[412512.0, 137168.0, 128.0, 2072.0, 551880.0, 0.75], [112429.0, 452804.0, 894.0, 5461.0, 571588.0, 0.79], [1744.0, 4452.0, 5572.0, 2293.0, 14061.0, 0.4], [3854.0, 8342.0, 4511.0, 38145.0, 54852.0, 0.7], [530539.0, 602766.0, 11105.0, 47971.0, 'mPA:', 0.66], [0.62, 0.63, 0.28, 0.59, 'mIoU:', 0.53], [0.78, 0.75, 0.5, 0.8, 'OA:', 0.76], [0.76, 0.77, 0.44, 0.74, 'F1-macro:', 0.68]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall   Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall             412512.0  137168.0    128.0     2072.0   551880.0   0.75
Rip Rap                          112429.0  452804.0    

In [61]:
# model_path = os.path.join(path_cur, 'trained_models', '200', 'model.pth')

model_path_2 = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_batch16/4class_16batch_unet_resnet50_0.001_ce_imagenet_CWFalse_(0.9, 0.999)-epoch=71-val_loss=0.54.ckpt"

gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')
labels = ['Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']

options = {'agg_to_water': True, 'batch': 32, 
          'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
          'model_path': model_path_2, 'predict_masks': False, 'gen_masks_path':gen_masks_path,
          'labels': labels}

main(options)

testing: 100%|██████████| 23/23 [00:20<00:00,  1.11it/s]


Evaluation: {'macroPrec': 0.7190646908035663, 'microPrec': 0.7795550165796976, 'weightPrec': 0.7807557341179913, 'macroRec': 0.636274585091505, 'microRec': 0.7795550165796976, 'weightRec': 0.7795550165796976, 'macroF1': 0.6704098950463839, 'microF1': 0.7795550165796976, 'weightF1': 0.778857947566291, 'subsetAcc': 0.7795550165796976, 'IoU': 0.5239488492823632}
[[480981.0, 133517.0, 386.0, 607.0, 615491.0, 0.78], [107422.0, 448946.0, 566.0, 3066.0, 560000.0, 0.8], [348.0, 5700.0, 4454.0, 2699.0, 13201.0, 0.34], [3037.0, 11913.0, 3774.0, 31147.0, 49871.0, 0.62], [591788.0, 600076.0, 9180.0, 37519.0, 'mPA:', 0.64], [0.66, 0.63, 0.25, 0.55, 'mIoU:', 0.52], [0.81, 0.75, 0.49, 0.83, 'OA:', 0.78], [0.8, 0.77, 0.4, 0.71, 'F1-macro:', 0.67]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall   Rip Rap   Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall             480981.0  133517.0   386.0      607.0   615491.0   0.78
Rip Rap                          107422.0  448946.0   566.0 

In [63]:
# Check training accuracy


model_dir = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_batch32"
model_files = [file for file in os.listdir(model_dir) if file.split(".")[-1]=="ckpt" and file.split("_")[0]=="4class"]

gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')

for file in model_files:
    print("Using model {}".format(file))
    model_path = os.path.join(model_dir, file)
    labels = ['Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
    options = {'agg_to_water': True, 'batch': 32, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
    try:
        main(options)
    except:
        labels = ['Background', 'Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
        options = {'agg_to_water': True, 'batch': 32, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
        main(options)


Load test set to memory:   0%|          | 0/756 [00:00<?, ?it/s]

Using model 4class_unet_resnet50_0.0001_ce_imagenet_CWFalse_(0.9, 0.999)-epoch=20-val_loss=0.56.ckpt


testing: 100%|██████████| 23/23 [00:17<00:00,  1.28it/s]


Evaluation: {'macroPrec': 0.6764536199817023, 'microPrec': 0.7516500977342291, 'weightPrec': 0.7651706144324054, 'macroRec': 0.6474145292343492, 'microRec': 0.7516500977342291, 'weightRec': 0.7516500977342291, 'macroF1': 0.6556088093833564, 'microF1': 0.7516500977342292, 'weightF1': 0.7508075058979647, 'subsetAcc': 0.7516500977342291, 'IoU': 0.5035659306172153}


Load test set to memory:   1%|▏         | 11/756 [00:00<00:06, 107.61it/s]

...

Evaluation: {'macroPrec': 0.5478349875915578, 'microPrec': 0.7346231075851611, 'weightPrec': 0.755513822900066, 'macroRec': 0.4904948706613377, 'microRec': 0.7346231075851611, 'weightRec': 0.7346231075851611, 'macroF1': 0.5076739489864118, 'microF1': 0.7346231075851611, 'weightF1': 0.7312546256869518, 'subsetAcc': 0.7346231075851611, 'IoU': 0.3875296811124339}


Load test set to memory:   2%|▏         | 12/756 [00:00<00:06, 116.46it/s]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [9.0, 386079.0, 226556.0, 87.0, 2760.0, 615491.0, 0.63], [0.0, 69465.0, 486023.0, 398.0, 4114.0, 560000.0, 0.87], [0.0, 43.0, 5370.0, 3588.0, 4200.0, 13201.0, 0.27], [0.0, 4079.0, 8117.0, 3488.0, 34187.0, 49871.0, 0.69], [9.0, 459666.0, 726066.0, 7561.0, 45261.0, 'mPA:', 0.49], [0.0, 0.56, 0.61, 0.21, 0.56, 'mIoU:', 0.39], [0.0, 0.84, 0.67, 0.47, 0.76, 'OA:', 0.73], [0.0, 0.72, 0.76, 0.35, 0.72, 'F1-macro:', 0.51]]


testing: 100%|██████████| 23/23 [00:16<00:00,  1.39it/s]


Evaluation: {'macroPrec': 0.5478349875915578, 'microPrec': 0.7346231075851611, 'weightPrec': 0.755513822900066, 'macroRec': 0.4904948706613377, 'microRec': 0.7346231075851611, 'weightRec': 0.7346231075851611, 'macroF1': 0.5076739489864118, 'microF1': 0.7346231075851611, 'weightF1': 0.7312546256869518, 'subsetAcc': 0.7346231075851611, 'IoU': 0.3875296811124339}
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [9.0, 386079.0, 226556.0, 87.0, 2760.0, 615491.0, 0.63], [0.0, 69465.0, 486023.0, 398.0, 4114.0, 560000.0, 0.87], [0.0, 43.0, 5370.0, 3588.0, 4200.0, 13201.0, 0.27], [0.0, 4079.0, 8117.0, 3488.0, 34187.0, 49871.0, 0.69], [9.0, 459666.0, 726066.0, 7561.0, 45261.0, 'mPA:', 0.49], [0.0, 0.56, 0.61, 0.21, 0.56, 'mIoU:', 0.39], [0.0, 0.84, 0.67, 0.47, 0.76, 'OA:', 0.73], [0.0, 0.72, 0.76, 0.35, 0.72, 'F1-macro:', 0.51]]
Confusion Matrix:  
                     Background Bulkhead Or Sea Wall   Rip Rap   Groin Breakwater        Sum Recall
Background                  0.0                  0.0       0

In [52]:
# Check training accuracy


model_dir = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_batch16"
model_files = [file for file in os.listdir(model_dir) if file.split(".")[-1]=="ckpt" and file.split("_")[0]=="4class"]

gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')

for file in model_files:
    print("Using model {}".format(file))
    model_path = os.path.join(model_dir, file)
    labels = ['Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
    options = {'agg_to_water': True, 'batch': 16, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
    try:
        main(options)
    except:
        labels = ['Background', 'Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
        options = {'agg_to_water': True, 'batch': 16, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
        main(options)


Load test set to memory:   2%|▏         | 17/756 [00:00<00:04, 168.94it/s]

Using model 4class_16batch_unet_resnet50_0.001_ce_imagenet_CWFalse_(0.9, 0.999)-epoch=71-val_loss=0.54.ckpt


testing: 100%|██████████| 45/45 [00:34<00:00,  1.32it/s]


Evaluation: {'macroPrec': 0.7190646908035663, 'microPrec': 0.7795550165796976, 'weightPrec': 0.7807557341179913, 'macroRec': 0.636274585091505, 'microRec': 0.7795550165796976, 'weightRec': 0.7795550165796976, 'macroF1': 0.6704098950463839, 'microF1': 0.7795550165796976, 'weightF1': 0.778857947566291, 'subsetAcc': 0.7795550165796976, 'IoU': 0.5239488492823632}


Load test set to memory:   2%|▏         | 13/756 [00:00<00:06, 122.43it/s]

...

Evaluation: {'macroPrec': 0.7345273987146328, 'microPrec': 0.7735997280719673, 'weightPrec': 0.7755721007347639, 'macroRec': 0.6614421789768543, 'microRec': 0.7735997280719673, 'weightRec': 0.7735997280719673, 'macroF1': 0.6926469665718991, 'microF1': 0.7735997280719673, 'weightF1': 0.7733143552969085, 'subsetAcc': 0.7735997280719673, 'IoU': 0.5427615916384809}


Load test set to memory:   2%|▏         | 14/756 [00:00<00:05, 139.65it/s]

[[472138.0, 143029.0, 26.0, 298.0, 615491.0, 0.77], [108140.0, 447500.0, 840.0, 3520.0, 560000.0, 0.8], [388.0, 4623.0, 5517.0, 2673.0, 13201.0, 0.42], [4085.0, 9225.0, 3564.0, 32997.0, 49871.0, 0.66], [584751.0, 604377.0, 9947.0, 39488.0, 'mPA:', 0.66], [0.65, 0.62, 0.31, 0.59, 'mIoU:', 0.54], [0.81, 0.74, 0.55, 0.84, 'OA:', 0.77], [0.79, 0.77, 0.48, 0.74, 'F1-macro:', 0.69]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall   Rip Rap   Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall             472138.0  143029.0    26.0      298.0   615491.0   0.77
Rip Rap                          108140.0  447500.0   840.0     3520.0   560000.0    0.8
Groin                               388.0    4623.0  5517.0     2673.0    13201.0   0.42
Breakwater                         4085.0    9225.0  3564.0    32997.0    49871.0   0.66
Sum                              584751.0  604377.0  9947.0    39488.0       mPA:   0.66
IoU                                  0.65      0.62    0.31       

testing: 100%|██████████| 45/45 [00:25<00:00,  1.74it/s]


Evaluation: {'macroPrec': 0.7215599338995531, 'microPrec': 0.766925864893429, 'weightPrec': 0.7795184566149642, 'macroRec': 0.666685592455204, 'microRec': 0.766925864893429, 'weightRec': 0.766925864893429, 'macroF1': 0.687525131652307, 'microF1': 0.766925864893429, 'weightF1': 0.7660058969562188, 'subsetAcc': 0.766925864893429, 'IoU': 0.5371723948639944}
[[428865.0, 184337.0, 271.0, 2018.0, 615491.0, 0.7], [74125.0, 480557.0, 1309.0, 4009.0, 560000.0, 0.86], [89.0, 4364.0, 5394.0, 3354.0, 13201.0, 0.41], [1821.0, 9972.0, 3008.0, 35070.0, 49871.0, 0.7], [504900.0, 679230.0, 9982.0, 44451.0, 'mPA:', 0.67], [0.62, 0.63, 0.3, 0.59, 'mIoU:', 0.54], [0.85, 0.71, 0.54, 0.79, 'OA:', 0.77], [0.77, 0.78, 0.47, 0.74, 'F1-macro:', 0.69]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall   Rip Rap   Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall             428865.0  184337.0   271.0     2018.0   615491.0    0.7
Rip Rap                           74125.0  480557.0  1309.0     40

In [62]:
# Check training accuracy


model_dir = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_batch8"
model_files = [file for file in os.listdir(model_dir) if file.split(".")[-1]=="ckpt" and file.split("_")[0]=="4class"]

gen_masks_path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test_masks')

for file in model_files:
    print("Using model {}".format(file))
    model_path = os.path.join(model_dir, file)
    labels = ['Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
    options = {'agg_to_water': True, 'batch': 16, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
    try:
        main(options)
    except:
        labels = ['Background', 'Bulkhead Or Sea Wall', 'Rip Rap', 'Groin', 'Breakwater']
        options = {'agg_to_water': True, 'batch': 16, 
              'input_channels': 4, 'output_channels': 5, 'hidden_channels': 16,
              'model_path': model_path, 'predict_masks': False, 'gen_masks_path':gen_masks_path, 'labels': labels}
        main(options)


Load test set to memory:   0%|          | 0/756 [00:00<?, ?it/s]

Using model 4class_unet_resnet50_0.001_ce_imagenet_CWFalse_(0.9, 0.999)-epoch=68-val_loss=0.57.ckpt


testing: 100%|██████████| 45/45 [00:18<00:00,  2.45it/s]


Evaluation: {'macroPrec': 0.7084187975860927, 'microPrec': 0.7648024363718277, 'weightPrec': 0.7652631198369233, 'macroRec': 0.6259063484321136, 'microRec': 0.7648024363718277, 'weightRec': 0.7648024363718277, 'macroF1': 0.658560142227529, 'microF1': 0.7648024363718278, 'weightF1': 0.7640485614289068, 'subsetAcc': 0.7648024363718277, 'IoU': 0.5108438512154319}


Load test set to memory:   1%|▏         | 10/756 [00:00<00:07, 98.45it/s]

...

Evaluation: {'macroPrec': 0.6797374152726499, 'microPrec': 0.7606032151775889, 'weightPrec': 0.7624940610134989, 'macroRec': 0.6151856557048266, 'microRec': 0.7606032151775889, 'weightRec': 0.7606032151775889, 'macroF1': 0.6424072408771212, 'microF1': 0.7606032151775889, 'weightF1': 0.7600833427641945, 'subsetAcc': 0.7606032151775889, 'IoU': 0.4950001614920939}


Load test set to memory:   1%|          | 9/756 [00:00<00:08, 88.93it/s]

[[465733.0, 149260.0, 0.0, 498.0, 615491.0, 0.76], [112710.0, 442017.0, 1126.0, 4147.0, 560000.0, 0.79], [450.0, 5717.0, 4073.0, 2961.0, 13201.0, 0.31], [3384.0, 11096.0, 5159.0, 30232.0, 49871.0, 0.61], [582277.0, 608090.0, 10358.0, 37838.0, 'mPA:', 0.62], [0.64, 0.61, 0.21, 0.53, 'mIoU:', 0.5], [0.8, 0.73, 0.39, 0.8, 'OA:', 0.76], [0.78, 0.76, 0.35, 0.69, 'F1-macro:', 0.64]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall   Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall             465733.0  149260.0      0.0      498.0   615491.0   0.76
Rip Rap                          112710.0  442017.0   1126.0     4147.0   560000.0   0.79
Groin                               450.0    5717.0   4073.0     2961.0    13201.0   0.31
Breakwater                         3384.0   11096.0   5159.0    30232.0    49871.0   0.61
Sum                              582277.0  608090.0  10358.0    37838.0       mPA:   0.62
IoU                                  0.64      0.61     0.21

testing: 100%|██████████| 45/45 [00:18<00:00,  2.49it/s]


Evaluation: {'macroPrec': 0.7485951630558859, 'microPrec': 0.7824898693082225, 'weightPrec': 0.7826882852633816, 'macroRec': 0.6969101604928749, 'microRec': 0.7824898693082225, 'weightRec': 0.7824898693082225, 'macroF1': 0.7204109870244086, 'microF1': 0.7824898693082224, 'weightF1': 0.7822626006293997, 'subsetAcc': 0.7824898693082225, 'IoU': 0.5740625259973031}
[[495755.0, 118956.0, 31.0, 749.0, 615491.0, 0.81], [124943.0, 431002.0, 1209.0, 2846.0, 560000.0, 0.77], [744.0, 4036.0, 6503.0, 1918.0, 13201.0, 0.49], [4028.0, 6183.0, 3757.0, 35903.0, 49871.0, 0.72], [625470.0, 560177.0, 11500.0, 41416.0, 'mPA:', 0.7], [0.67, 0.63, 0.36, 0.65, 'mIoU:', 0.57], [0.79, 0.77, 0.57, 0.87, 'OA:', 0.78], [0.8, 0.77, 0.53, 0.79, 'F1-macro:', 0.72]]
Confusion Matrix:  
                     Bulkhead Or Sea Wall   Rip Rap    Groin Breakwater        Sum Recall
Bulkhead Or Sea Wall             495755.0  118956.0     31.0      749.0   615491.0   0.81
Rip Rap                          124943.0  431002.0   1

In [ ]:
# 4class_unet_resnet18_0.001_ce_swsl-epoch=51-val_loss=0.61.ckpt failed prediction
# 4class_unet_resnet18_0.001_ce_ssl-epoch=63-val_loss=0.59.ckpt failed prediction
# 4class_unet_resnet18_0.001_ce_imagenet-epoch=43-val_loss=0.58.ckpt failed prediction

# 4class_unet_resnet50_0.001_ce_swsl-epoch=56-val_loss=0.56.ckpt failed prediction
# 4class_unet_resnet50_0.001_ce_ssl-epoch=51-val_loss=0.60.ckpt failed prediction
# 4class_unet_resnet50_0.001_ce_imagenet-epoch=50-val_loss=0.57.ckpt failed prediction

# 4class_unet_resnet18_0.001_focal_imagenet-epoch=13-val_loss=0.00.ckpt failed prediction
# 4class_unet_resnet50_0.001_focal_imagenet-epoch=21-val_loss=0.00.ckpt failed prediction
# 4class_unet_resnet50_0.0001_focal_imagenet-epoch=14-val_loss=0.00.ckpt failed prediction




In [ ]:
model_path = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results/4class_unet_resnet50_0.001_ce_imagenet-epoch=50-val_loss=0.57.ckpt"

path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test')
ROIs = [file for file in os.listdir(path) if file.split('.')[-1]=='tif']

impute_nan = np.tile(bands_mean, (256,256,1))

# Load pretrained model
model = SemanticSegmentationTask.load_from_checkpoint(model_path)
transform_test = transforms.Compose([transforms.ToTensor()])

with torch.no_grad():
    model.eval()

    for roi in tqdm(ROIs[0:2]):

        roi_file = os.path.join(path, roi)
        os.makedirs(options['gen_masks_path'], exist_ok=True)

        output_image = os.path.join(options['gen_masks_path'], os.path.basename(roi_file).split('.tif')[0] + '_unet.tif')

        # Read metadata of the initial image
        with rasterio.open(roi_file, mode ='r') as src:
            tags = src.tags().copy()
            meta = src.meta
            image = src.read()
            image = np.moveaxis(image, (0, 1, 2), (2, 0, 1)).astype('float32')
            dtype = src.read(1).dtype


        # Update meta to reflect the number of layers
        meta.update(count = 1)

        # Write it
        with rasterio.open(output_image, 'w', **meta) as dst:

            nan_mask = np.isnan(image)
            image[nan_mask] = impute_nan[nan_mask]

    #         image = torch.from_numpy(image)
            image = transform_test(image)


            logits = model(image.unsqueeze(0))

            probs = logits.softmax(dim=1).numpy().argmax(1).squeeze()
            print(type(probs))

    #         probs = torch.nn.functional.softmax(logits.detach(), dim=1).cpu().numpy()
    #         probs = probs.argmax(1).squeeze()

            dst.write_band(1, probs.astype(dtype).copy()) # In order to be in the same dtype
            dst.update_tags(**tags)


    #                     # Preprocessing before prediction

    # #                     nan_mask = np.isnan(image)
    # #                     image[nan_mask] = impute_nan[nan_mask]

    #                     image = transform_test(image)

    # #                     image = standardization(image)

    # #                     # Image to Cuda if exist
    # #                     # image = image.to(device)

    #                     # Predictions
    #                     logits = model(image.unsqueeze(0))

    #                     probs = torch.nn.functional.softmax(logits.detach(), dim=1).cpu().numpy()

    #                     probs = probs.argmax(1).squeeze()+1

    #                     # Write the mask with georeference
    #                     dst.write_band(1, probs.astype(dtype).copy()) # In order to be in the same dtype
    #                     dst.update_tags(**tags)


    # # #         image = standardization(image)

    # # #                     # Image to Cuda if exist
    # # #                     # image = image.to(device)


    # #         logits = model(image.unsqueeze(0))
    # #         image = np.expand_dims(image, axis=0)

    #         logits = model(image)
    # #         pr_masks = logits.softmax(dim=1).argmax(1)

    # # #         probs = torch.nn.functional.softmax(logits.detach(), dim=1).cpu().numpy()

    # # #         probs = probs.argmax(1).squeeze() + 1

    # #         dst.write_band(1, pr_masks.astype(dtype).copy()) # In order to be in the same dtype
    # #         dst.update_tags(**tags)



In [ ]:
    
transform_test = transforms.Compose([transforms.ToTensor()])
standardization = transforms.Normalize(bands_mean, bands_std)
    
# Construct Test data loader

dataset_test = GenDEBRIS('test', transform=transform_test, standardization = standardization, agg_to_water = True)
test_loader = DataLoader(   dataset_test, 
                            batch_size = 32, 
                            shuffle = False)

# visualization and check on the predicted results

labels = ['Background', 'Bulkhead Or Sea Wall', 'Rip Rap', 
            'Groin', 'Breakwater']

path = "/rapids/notebooks/sciclone/geograd/Miranda/github/marine-debris.github.io/torchlightning_results_norm/4class_unet_resnet50_0.0001_jaccard_imagenet-epoch=75-val_loss=0.49.ckpt"

# Load pretrained model
model = SemanticSegmentationTask.load_from_checkpoint(path)

batch = next(iter(test_loader))
with torch.no_grad():
    model.eval()

    logits = model(batch["image"])
    target = batch["mask"]

    
# y_predicted = []
# y_true = []

pr_masks = logits.softmax(dim=1).argmax(1)
# logits = torch.nn.functional.softmax(logits, dim=1).argmax(1)


# logits = logits.reshape(-1)
# target = target.reshape(-1)

# mask = target != 0 # 0 is the background
            
# logits = logits[mask]
# target = target[mask]

# y_predicted += logits.tolist()
# y_true += target.tolist()



def formatter_func(x, pos):
    if x==0:
        return "Background"
    elif x==1:
        return "bulkhead"
    elif x==2:
        return "riprap"
    elif x==3:
        return "groin"
    elif x==4:
        return "breakwater"
    else:
        return ""

# Formatter
valid_values = range(0,5)

lookup = dict(zip(valid_values, labels))
# formatter = plt.FuncFormatter(lambda val, loc: lookup[val])
formatter = plt.FuncFormatter(formatter_func)


for image, gt_mask, pr_mask in zip(batch["image"], batch["mask"], pr_masks):
    
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(image.numpy().transpose(1, 2, 0)[:,:,0:3].astype(float))  # convert CHW -> HWC 
    plt.title("Image")
    plt.axis("off")
    

    plt.subplot(1, 3, 2)
    plt.imshow(gt_mask.numpy().squeeze(),cmap=plt.cm.get_cmap('rainbow', 5)) # just squeeze classes dim, because we have only one class
    plt.title("Ground truth")
    plt.colorbar(ticks=[0, 1, 2, 3, 4], format=formatter)

    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(pr_mask.numpy().squeeze(), cmap=plt.cm.get_cmap('rainbow', 5)) # just squeeze classes dim, because we have only one class
    plt.title("Prediction")
    plt.colorbar(ticks=[0,1,2,3,4], format=formatter)

    plt.axis("off")

    plt.show()


In [ ]:
from imgaugmentation import ImgAugTransform

In [ ]:
# Test the customized Semantic Segmentation trainer

from vims_dataloader import GenDEBRIS, RandomRotationTransform , class_distr, gen_weights, bands_mean, bands_std


# Set transformation for dataset

transform_train = transforms.Compose([ImgAugTransform(),
                                    transforms.ToTensor()])

# transforms.Compose([transforms.ToTensor(),
#                                     RandomRotationTransform([-90, 0, 90, 180]),
#                                     transforms.RandomHorizontalFlip()])




transform_test = transforms.Compose([transforms.ToTensor()])

standardization = transforms.Normalize(bands_mean, bands_std)


dataset_train = GenDEBRIS('val', transform=transform_train, standardization = standardization, agg_to_water = True)
# dataset_val = GenDEBRIS('val', transform=transform_test, standardization = standardization, agg_to_water = True)
# dataset_test = GenDEBRIS('test', transform=transform_test, standardization = standardization, agg_to_water = True)

# Create data loaders



train_dataloader = DataLoader(dataset_train, 
                            batch_size = 32, 
                            shuffle = True,
                            num_workers = 0)

# val_dataloader = DataLoader(dataset_val, 
#                             batch_size = 32, 
#                             shuffle = False,
#                             num_workers=0)

# test_dataloader = DataLoader(dataset_test, 
#                             batch_size = 32, 
#                             shuffle = False,
#                             num_workers=0)

In [ ]:

# visualization of input dataset and masks    

def vis(batch):

    for image, gt_mask in zip(batch["image"], batch["mask"]):
    
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow(image.numpy().transpose(1, 2, 0)[:,:,0:3].astype(int))  # convert CHW -> HWC 
        plt.title("Image")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(gt_mask.numpy().squeeze()) # just squeeze classes dim, because we have only one class
        plt.title("Ground truth")
        plt.colorbar(ticks=[0,1,2,3,4])
        plt.axis("off")

        plt.show()


In [ ]:
batch = next(iter(train_dataloader))
# vis(batch)

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger

model = SemanticSegmentation(
                segmentation_model="unet",
                encoder_name="resnet18",
                encoder_weights="imagenet",
                learning_rate=1e-4,
                in_channels=4,
                num_classes=5,
                learning_rate_schedule_patience=10,
                ignore_zeros=False,
                loss="focal",
                imagenet_pretraining=True,
                class_weights=None) # None or a tensor of weights


# Set the output directory and callbacks
experiment_dir = os.path.join(root_path, "delete_testSemanticSegmentationTrainer")

checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=experiment_dir,
        save_top_k=1,
        filename="testing-{epoch:02d}-{val_loss:.2f}",
        save_last=True)

early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        min_delta=0.00,
        patience=50)

csv_logger = CSVLogger(
        save_dir=experiment_dir,
        name="testing_csv")


trainer = pl.Trainer(
        callbacks=[checkpoint_callback, early_stopping_callback],
        logger=[csv_logger],
        default_root_dir=experiment_dir,
        min_epochs=1,
        max_epochs=1000,
        gpus=[3])
    
    




In [ ]:
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
#     checkpoint_callback.best_model_path
    
trainer.test(ckpt_path="best", dataloaders=test_dataloader)

In [ ]:
import itertools

training_set_options = ["4class"]
model_options = ["unet"]
encoder_options = ["resnet18", "resnet50"]
lr_options = [1e-4, 1e-3]
loss_options = ["focal"]
weight_init_options = ["imagenet"]
in_channel = 4
out_channel = 5 # including background

count = 0

for (train_state, model, encoder, lr, loss, weight_init) in itertools.product(
        training_set_options,
        model_options,
        encoder_options,
        lr_options,
        loss_options,
        weight_init_options):
    
    count += 1

In [ ]:
#### TRANSFORM DATA ####

class ImgAugTransform:

  def __init__(self):

    self.aug = iaa.Sequential([

        iaa.Scale((224, 224)),

        iaa.Sometimes(0.30, iaa.GaussianBlur(sigma=(0, 3.0))),

                iaa.Sometimes(0.25, iaa.Multiply((0.5, 1.5), per_channel=0.5)),

                iaa.Sometimes(0.20, iaa.Invert(0.25, per_channel=0.5)),

                iaa.Sometimes(0.25, iaa.ReplaceElementwise(

                    iap.FromLowerResolution(iap.Binomial(0.1), size_px=8),

                    iap.Normal(128, 0.4*128),

                    per_channel=0.5)

                                         ),

                iaa.Sometimes(0.30, iaa.AdditivePoissonNoise(40)),

        iaa.Fliplr(0.5),

        iaa.Affine(rotate=(-20, 20), mode='symmetric'),

        iaa.Sometimes(0.30,

                      iaa.OneOf([iaa.Dropout(p=(0, 0.1)),

                                 iaa.CoarseDropout(0.1, size_percent=0.5)])),

        iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)

    ])

      

  def __call__(self, img):

    img = np.array(img)

    return self.aug.augment_image(img)




    

data_transforms = {

    'train': transforms.Compose([

            ImgAugTransform(),

        transforms.ToTensor(),

        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ]),

    'val': transforms.Compose([

            ImgAugTransform(),

        transforms.ToTensor(),

        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ]),

}